In [ ]:
# This file is used by get_new_raw_file.py to create a report after checking a new download.  To make this work nicely,
# you should use the 'hide-input-all' nbextension and before get_new_raw_file, enable hide all, reset and clear
# all cells, save the sheet, and CLose and Halt.  

In [ ]:
from math import log10, floor
def round_sig(x, sig=2):
    if abs(x)>=1:
        out =  int(round(x, sig-int(floor(log10(abs(x))))-1))
        return f"{out:,d}" # does the right thing with commas
    else: # fractional numbers
        return str(round(x, sig-int(floor(log10(abs(x))))-1))
        

In [ ]:
#preamble to analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.ticker
from IPython.display import Markdown as md
from time import sleep
from datetime import datetime
today = str(datetime.today())
today = today.split()[0]

In [ ]:
md(f'# FracFocus bulk download summary report for {today}')

In [ ]:
sources = './sources/'
datefn= './out/upload_dates.csv'
currfn = 'testData'
outdir = './out/'
tempfolder = './tmp/'
webworkfolder = './website_gen/'
zfilename = 'testData'

updates = pd.read_csv(datefn)
new_upk = updates[updates.date_added==today].UploadKey.tolist()
#print(new_upk)
md(f'### Number of new disclosures added today: {len(new_upk)}')


In [ ]:
import core.Construct_set as const_set
t = const_set.Construct_set(fromScratch=False,
                            zfilename=zfilename,
                            sources=sources,
                            outdir=outdir,
                            tempfolder=tempfolder).get_full_set();
df = t.tables['allrec'].get_df()
locat = t.get_df_location()[['UploadKey','StateName','CountyName',
                             'iOperatorName','iUploadKey','TotalBaseWaterVolume']]
#print(len(df), df.columns)
df = pd.merge(df,locat,on='iUploadKey',how='left')
#print(len(df), df.head())
df = df[df.UploadKey.isin(new_upk)]
#print(len(df), df.columns)
opdf = t.tables['operator'].get_df()
df = pd.merge(df,opdf,on='iOperatorName',how='left')

In [ ]:
md("""The following list is the most recently published fracking events. Note that these are still
raw names and numbers and have not yet been checked for validity.""")

In [ ]:
gb = df.groupby(['UploadKey','StateName'],as_index=False)[['CountyName',
                                                           'OperatorName',
                                                  'TotalBaseWaterVolume']].first()
tmp1 = gb.groupby(['StateName','CountyName','OperatorName'],as_index=False)['UploadKey'].count()
tmp1.rename({'UploadKey':'Disclosure_cnt'},inplace=True,axis=1)
tmp2 = gb.groupby(['StateName','CountyName','OperatorName'],
          as_index=False)['TotalBaseWaterVolume'].mean()
tmp2.rename({'TotalBaseWaterVolume':'mean_Water_Used'},axis=1,inplace=True)
tmp2.mean_Water_Used = tmp2.mean_Water_Used.map(lambda x: round_sig(x,3))
out = pd.merge(tmp1,tmp2,on=['StateName','CountyName','OperatorName'],how='left')

out